In [4]:
SELECT COUNT (*)
  FROM [ERPDB].[etl].[vwJobMtlChange]

(1 row affected)

Total execution time: 00:00:14.581

(No column name)
592795


In [2]:
SELECT COUNT (*)
  FROM [ERPDB].[Erp].[JobMtl]

(1 row affected)

Total execution time: 00:00:00.126

(No column name)
592795


In [3]:
Select Count (*)
From(
SELECT
jm.*, cl.DateStampedOn, cc.Calculated_ChangeDate, od.ProjectID,
 (case when
            cc.Calculated_ChangeDate >= cl.DateStampedOn
            then cc.Calculated_ChangeDate
            else 
            cl.DateStampedON
            end) as Latest_ChangeDate
,Dateadd(second, 00, cast(
    (case when
            cc.Calculated_ChangeDate >= cl.DateStampedOn
            then cc.Calculated_ChangeDate
            else 
            cl.DateStampedON
            end)as Datetime)) as Latest_ChangeDateTime
FROM
(
    SELECT JobMtl.*,
    REVERSE(PARSENAME(REPLACE(REVERSE([JobNum]), '-', '.'), 1)) as OrderNum,
    REVERSE(PARSENAME(REPLACE(REVERSE([JobNum]), '-', '.'), 2)) as OrderLine,
    (Case
      When
        REVERSE(PARSENAME(REPLACE(REVERSE([JobNum]), '-', '.'), 2)) is Null
      Then
        REVERSE(PARSENAME(REPLACE(REVERSE([JobNum]), '-', '.'), 1))
      Else
        CONCAT(REVERSE(PARSENAME(REPLACE(REVERSE([JobNum]), '-', '.'), 1))
        ,'-'
        ,REVERSE(PARSENAME(REPLACE(REVERSE([JobNum]), '-', '.'), 2))
        )
    end) as OrderNumLine,
    CONCAT(JobNum,'~',AssemblySeq,'~',MtlSeq) as ForeignKey2
    FROM [ERPDB].[Erp].[JobMtl]
) jm

-- Get the DateStampedOn from the ChangeLog
LEFT JOIN
(
Select *
From
    (Select DateStampedOn,
            Key1,
            Key2,
            LogText,
            ROW_NUMBER() OVER(Partition by Key2 Order by DateStampedON Desc) rn
        From [ERPDB].[Ice].[ChgLog] 
    ) a
    Where rn=1 
) cl
ON jm.JobNum=cl.Key1 and jm.ForeignKey2=cl.Key2


--get the calculated change date from PartTran and Job Head Tables 
Left Join
(
Select  *
From 
    (Select 
        f.Company,
        f.JobNum,
        f.AssemblySeq,
        f.JobSeq,
        f.PartNum,
        f.TranDate,
        f.CreateDate,
        f.CloseDate,
        f.ForeignKey2,
        f.Calculated_ChangeDate,
        ROW_NUMBER() OVER(PARTITION By ForeignKey2 ORDER BY f.Calculated_ChangeDate DESC) rn
    FROM
        (SELECT 
            pt.Company as Company, 
            pt.JobNum as JobNum,
            pt.AssemblySeq as AssemblySeq,
            pt.JobSeq as JobSeq,
            pt.TranDate as TranDate,
            pt.PartNum as PartNum,
            jh.CreateDate as CreateDate,
            jh.ClosedDate as CloseDate,
            pt.ForeignKey2 as ForeignKey2,
            (case when
            pt.TranDate >= jh.CreateDate
            then
            pt.TranDate
            when
            jh.CreateDate >= jh.ClosedDate
            then
            jh.CreateDate
            else
            jh.ClosedDate
            end) as Calculated_ChangeDate
                FROM
                (SELECT
                    Company,
                    JobNum,
                    AssemblySeq,
                    JobSeq,
                    TranDate,
                    PartNum,
                    CONCAT(JobNum,'~',AssemblySeq,'~',JobSeq) as ForeignKey2
                    FROM [ERPDB].[Erp].[PartTran]
                ) as pt
                Left Join
                (SELECT 
                    Company, 
                    ClosedDate, 
                    JobNum,
                    CreateDate 
                    FROM [ERPDB].[Erp].[JobHead]
                ) as jh
                    on pt.Company = jh.Company AND pt.JobNum = jh.JobNum
        ) f 
    ) b
    where rn = 1
) cc
on jm.ForeignKey2 = cc.ForeignKey2
Left Join [etl].[vwOrderDetail] AS od
on jm.OrderNumLine = od.OrderNumLine and jm.company = od.company
) jmchange

(1 row affected)

Total execution time: 00:00:10.630

(No column name)
592795


In [4]:
Select Top (10) *
From [ERPDB].[etl].[vwJobMtlChange]
Where JobNum like '2170022%' and Company = 'Solar'
Order By Latest_ChangeDate DESC


(30 rows affected)

Total execution time: 00:00:07.587

Company,JobComplete,IssuedComplete,JobNum,AssemblySeq,MtlSeq,PartNum,Description,QtyPer,RequiredQty,IUM,LeadTime,RelatedOperation,MtlBurRate,EstMtlBurUnitCost,EstUnitCost,IssuedQty,TotalCost,MtlBurCost,ReqDate,WarehouseCode,SalvagePartNum,SalvageDescription,SalvageQtyPer,SalvageUM,SalvageMtlBurRate,SalvageUnitCredit,SalvageEstMtlBurUnitCredit,SalvageQtyToDate,SalvageCredit,SalvageMtlBurCredit,MfgComment,VendorNum,PurPoint,BuyIt,Ordered,PurComment,BackFlush,EstScrap,EstScrapType,FixedQty,FindNum,RevisionNum,SndAlrtCmpl,RcvInspectionReq,Plant,Direct,MaterialMtlCost,MaterialLabCost,MaterialSubCost,MaterialBurCost,SalvageMtlCredit,SalvageLbrCredit,SalvageBurCredit,SalvageSubCredit,APSAddResType,CallNum,CallLine,ProdCode,UnitPrice,BillableUnitPrice,DocBillableUnitPrice,ResReasonCode,PricePerCode,Billable,ShippedQty,DocUnitPrice,QtyStagedToDate,AddedMtl,MiscCharge,MiscCode,SCMiscCode,RFQNeeded,RFQVendQuotes,RFQNum,RFQLine,RFQStat,AnalysisCode,GlbRFQ,WhseAllocFlag,WIReqDate,Rpt1BillableUnitPrice,Rpt2BillableUnitPrice,Rpt3BillableUnitPrice,Rpt1UnitPrice,Rpt2UnitPrice,Rpt3UnitPrice,BaseRequiredQty,BaseUOM,Weight,WeightUOM,ReqRefDes,BasePartNum,BaseRevisionNum,SelectForPicking,StagingWarehouseCode,StagingBinNum,PickError,EstMtlUnitCost,EstLbrUnitCost,EstBurUnitCost,EstSubUnitCost,SalvageEstMtlUnitCredit,SalvageEstLbrUnitCredit,SalvageEstBurUnitCredit,SalvageEstSubUnitCredit,LoanedQty,BorrowedQty,ReassignSNAsm,GeneralPlanInfo,EstStdDescription,PricingUOM,RemovedFromPlan,IsPOCostingMaintained,EstStdType,POCostingFactor,PlannedQtyPerUnit,POCostingDirection,POCostingUnitVal,GroupSeq,SysRevID,SysRowID,OrigStructTag,OrigGroupSeq,ShowStatusIcon,ContractID,LinkToContract,StagingLotNum,PCLinkRemoved,ExternalMESSyncRequired,ExternalMESLastSync,LocationView,AttributeSetID,PlanningNumberOfPieces,SalvageAttributeSetID,SalvagePlanningNumberOfPieces,SalvagePlanningAttributeSetID,ForeignKey2,DateStampedOn,Calculated_ChangeDate,Latest_ChangeDate,Latest_ChangeDateTime
SOLAR,0,1,2170022-2-1,0,10,MFG MATERIAL,MFG MATERIAL,1.00000000,1.00000000,EA,0,0,0.00000,0.00000,0.00000,1.00000000,3875930.000,0.000,2021-03-12,,,,0.00000000,,0.00000,0.00000,0.00000,0.00000000,0.000,0.000,,0,,0,0,,0,0.00,%,1,,A,0,0,MfgSys,0,3875930.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,M,0,0,MFG,0.00000,0.00000,0.00000,,E,0,0.00000000,0.00000,0.00000000,0,1,MISC,,0,0,0,0,,,0,0,2021-03-12,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000000,EA,0.00000,,0,,,0,,,,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00,0.00,0,,,,0,0,0,0.00000,0.00000,0,0.00000,0,0x000000032699B206,b749bb08-cd50-41e2-b8b5-ac9aa7ee8567,,0,,,0,,0,1,NULL,0,0,0,0,0,0,2170022-2-1~0~10,2021-06-17,2022-07-15,2022-07-15,2022-07-15 00:00:00.000
SOLAR,0,1,2170022-5-1,0,250,250000,Tri County Asphalt,1.00000000,1.00000000,EA,0,0,0.00000,0.00000,0.00000,1.00000000,15000.000,0.000,2021-03-12,,,,0.00000000,,0.00000,0.00000,0.00000,0.00000000,0.000,0.000,,0,,0,0,,0,0.00,%,1,,,0,0,MfgSys,0,15000.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,M,0,0,,0.00000,0.00000,0.00000,,E,0,0.00000000,0.00000,0.00000000,0,1,MISC,,0,0,0,0,,,0,0,2021-03-12,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000000,EA,0.00000,,0,,,0,,,,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00,0.00,0,,,,0,0,0,0.00000,0.00000,0,0.00000,0,0x0000000325D4E986,4dc1280a-757d-4c31-8c63-f30c574ed513,,0,,,0,,0,1,NULL,0,0,0,0,0,0,2170022-5-1~0~250,2022-07-11,2022-07-14,2022-07-14,2022-07-14 00:00:00.000
SOLAR,0,1,2170022-5-1,0,260,260000,Sherwin Williams,1.00000000,1.00000000,EA,0,0,0.00000,0.00000,0.00000,1.00000000,1168.860,0.000,2021-03-12,,,,0.00000000,,0.00000,0.00000,0.00000,0.00000000,0.000,0.000,,0,,0,0,,0,0.00,%,1,,,0,0,MfgSys,0,1168.860,0.000,0.000,0.000,0.000,0.000,0.000,0.000,M,0,0,,0.00000,0.00000,0.00000,,E,0,0.00000000,0.00000,0.00000000,0,1,MISC,,0,0,0,0,,,0,0,2021-03-12,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000000,EA,0.00000,,0,,,0,,,,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00

In [14]:
---Final sample code for view creation

SELECT jm.*, cl.DateStampedOn, cc.Calculated_ChangeDate,
 (case when
            cc.Calculated_ChangeDate >= cl.DateStampedOn
            then cc.Calculated_ChangeDate
            else 
            cl.DateStampedON
            end) as Latest_ChangeDate
,Dateadd(second, 00, cast(
    (case when
            cc.Calculated_ChangeDate >= cl.DateStampedOn
            then cc.Calculated_ChangeDate
            else 
            cl.DateStampedON
            end)as Datetime)) as Latest_ChangeDateTime
FROM
(
    SELECT JobMtl.*,
    CONCAT(JobNum,'~',AssemblySeq,'~',MtlSeq) as ForeignKey2
    FROM [ERPDB].[Erp].[JobMtl]
) jm

-- Get the DateStampedOn from the ChangeLog
LEFT JOIN
(
Select *
From
    (Select DateStampedOn,
            Key1,
            Key2,
            LogText,
            ROW_NUMBER() OVER(Partition by Key2 Order by DateStampedON Desc) rn
        From [ERPDB].[Ice].[ChgLog] 
    ) a
    Where rn=1 
) cl
ON jm.JobNum=cl.Key1 and jm.ForeignKey2=cl.Key2


--get the calculated change date from PartTran and Job Head Tables 
Left Join
(
Select  *
From 
    (Select 
        f.Company,
        f.JobNum,
        f.AssemblySeq,
        f.JobSeq,
        f.PartNum,
        f.TranDate,
        f.CreateDate,
        f.CloseDate,
        f.ForeignKey2,
        f.Calculated_ChangeDate,
        ROW_NUMBER() OVER(PARTITION By ForeignKey2 ORDER BY f.Calculated_ChangeDate DESC) rn
    FROM
        (SELECT 
            pt.Company as Company, 
            pt.JobNum as JobNum,
            pt.AssemblySeq as AssemblySeq,
            pt.JobSeq as JobSeq,
            pt.TranDate as TranDate,
            pt.PartNum as PartNum,
            jh.CreateDate as CreateDate,
            jh.ClosedDate as CloseDate,
            pt.ForeignKey2 as ForeignKey2,
            (case when
            pt.TranDate >= jh.CreateDate
            then
            pt.TranDate
            when
            jh.CreateDate >= jh.ClosedDate
            then
            jh.CreateDate
            else
            jh.ClosedDate
            end) as Calculated_ChangeDate
                FROM
                (SELECT
                    Company,
                    JobNum,
                    AssemblySeq,
                    JobSeq,
                    TranDate,
                    PartNum,
                    CONCAT(JobNum,'~',AssemblySeq,'~',JobSeq) as ForeignKey2
                    FROM [ERPDB].[Erp].[PartTran]
                ) as pt
                Left Join
                (SELECT 
                    Company, 
                    ClosedDate, 
                    JobNum,
                    CreateDate 
                    FROM [ERPDB].[Erp].[JobHead]
                ) as jh
                    on pt.Company = jh.Company AND pt.JobNum = jh.JobNum
                WHERE pt.Jobnum like '2170022%' and pt.Company = 'Solar'
        ) f 
    ) b
    where rn = 1
) cc
on jm.ForeignKey2 = cc.ForeignKey2

WHERE jm.Jobnum like '2170022%' and jm.Company = 'Solar'
Order By Latest_ChangeDate DESC

(30 rows affected)

Total execution time: 00:00:03.796

Company,JobComplete,IssuedComplete,JobNum,AssemblySeq,MtlSeq,PartNum,Description,QtyPer,RequiredQty,IUM,LeadTime,RelatedOperation,MtlBurRate,EstMtlBurUnitCost,EstUnitCost,IssuedQty,TotalCost,MtlBurCost,ReqDate,WarehouseCode,SalvagePartNum,SalvageDescription,SalvageQtyPer,SalvageUM,SalvageMtlBurRate,SalvageUnitCredit,SalvageEstMtlBurUnitCredit,SalvageQtyToDate,SalvageCredit,SalvageMtlBurCredit,MfgComment,VendorNum,PurPoint,BuyIt,Ordered,PurComment,BackFlush,EstScrap,EstScrapType,FixedQty,FindNum,RevisionNum,SndAlrtCmpl,RcvInspectionReq,Plant,Direct,MaterialMtlCost,MaterialLabCost,MaterialSubCost,MaterialBurCost,SalvageMtlCredit,SalvageLbrCredit,SalvageBurCredit,SalvageSubCredit,APSAddResType,CallNum,CallLine,ProdCode,UnitPrice,BillableUnitPrice,DocBillableUnitPrice,ResReasonCode,PricePerCode,Billable,ShippedQty,DocUnitPrice,QtyStagedToDate,AddedMtl,MiscCharge,MiscCode,SCMiscCode,RFQNeeded,RFQVendQuotes,RFQNum,RFQLine,RFQStat,AnalysisCode,GlbRFQ,WhseAllocFlag,WIReqDate,Rpt1BillableUnitPrice,Rpt2BillableUnitPrice,Rpt3BillableUnitPrice,Rpt1UnitPrice,Rpt2UnitPrice,Rpt3UnitPrice,BaseRequiredQty,BaseUOM,Weight,WeightUOM,ReqRefDes,BasePartNum,BaseRevisionNum,SelectForPicking,StagingWarehouseCode,StagingBinNum,PickError,EstMtlUnitCost,EstLbrUnitCost,EstBurUnitCost,EstSubUnitCost,SalvageEstMtlUnitCredit,SalvageEstLbrUnitCredit,SalvageEstBurUnitCredit,SalvageEstSubUnitCredit,LoanedQty,BorrowedQty,ReassignSNAsm,GeneralPlanInfo,EstStdDescription,PricingUOM,RemovedFromPlan,IsPOCostingMaintained,EstStdType,POCostingFactor,PlannedQtyPerUnit,POCostingDirection,POCostingUnitVal,GroupSeq,SysRevID,SysRowID,OrigStructTag,OrigGroupSeq,ShowStatusIcon,ContractID,LinkToContract,StagingLotNum,PCLinkRemoved,ExternalMESSyncRequired,ExternalMESLastSync,LocationView,AttributeSetID,PlanningNumberOfPieces,SalvageAttributeSetID,SalvagePlanningNumberOfPieces,SalvagePlanningAttributeSetID,ForeignKey2,DateStampedOn,Calculated_ChangeDate,Latest_ChangeDate,Latest_ChangeDateTime
SOLAR,0,1,2170022-5-1,0,140,14000,Trillium,1.00000000,1.00000000,EA,0,0,0.00000,0.00000,214500.00000,1.00000000,73989.000,0.000,2021-03-12,,,,0.00000000,,0.00000,0.00000,0.00000,0.00000000,0.000,0.000,,0,,0,0,,0,0.00,%,1,,,0,0,MfgSys,0,73989.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,M,0,0,,0.00000,0.00000,0.00000,,E,0,0.00000000,0.00000,0.00000000,0,1,MISC,,0,0,0,0,,,0,0,2021-03-12,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000000,EA,0.00000,,0,,,0,,,,214500.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00,0.00,0,,,,0,0,0,0.00000,0.00000,0,0.00000,0,0x0000000325D4E96E,0b2ae063-921f-41be-bb90-76519dbfabef,,0,Complete,,0,,0,1,NULL,0,0,0,0,0,0,2170022-5-1~0~140,2022-07-06,2022-07-14,2022-07-14,2022-07-14 00:00:00.000
SOLAR,0,1,2170022-5-1,0,250,250000,Tri County Asphalt,1.00000000,1.00000000,EA,0,0,0.00000,0.00000,0.00000,1.00000000,15000.000,0.000,2021-03-12,,,,0.00000000,,0.00000,0.00000,0.00000,0.00000000,0.000,0.000,,0,,0,0,,0,0.00,%,1,,,0,0,MfgSys,0,15000.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,M,0,0,,0.00000,0.00000,0.00000,,E,0,0.00000000,0.00000,0.00000000,0,1,MISC,,0,0,0,0,,,0,0,2021-03-12,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000000,EA,0.00000,,0,,,0,,,,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00,0.00,0,,,,0,0,0,0.00000,0.00000,0,0.00000,0,0x0000000325D4E986,4dc1280a-757d-4c31-8c63-f30c574ed513,,0,,,0,,0,1,NULL,0,0,0,0,0,0,2170022-5-1~0~250,2022-07-11,2022-07-14,2022-07-14,2022-07-14 00:00:00.000
SOLAR,0,1,2170022-5-1,0,260,260000,Sherwin Williams,1.00000000,1.00000000,EA,0,0,0.00000,0.00000,0.00000,1.00000000,1168.860,0.000,2021-03-12,,,,0.00000000,,0.00000,0.00000,0.00000,0.00000000,0.000,0.000,,0,,0,0,,0,0.00,%,1,,,0,0,MfgSys,0,1168.860,0.000,0.000,0.000,0.000,0.000,0.000,0.000,M,0,0,,0.00000,0.00000,0.00000,,E,0,0.00000000,0.00000,0.00000000,0,1,MISC,,0,0,0,0,,,0,0,2021-03-12,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000000,EA,0.00000,,0,,,0,,,,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.0

**<mark>Use this for part of code -</mark> ** This formula will get the JobMtl Table joined to the most recent DateStamped on Date for each rob of the JobMtl Table

In [11]:
SELECT a.*, cl.* 
FROM
(
    SELECT JobMtl.*,
    CONCAT(JobNum,'~',AssemblySeq,'~',MtlSeq) as ForeignKey2
    FROM [ERPDB].[Erp].[JobMtl]
) a

LEFT JOIN
(
Select *
From
(Select DateStampedOn,
        Key1,
        Key2,
        LogText,
        ROW_NUMBER() OVER(Partition by Key2 Order by DateStampedON Desc) rn
    From [ERPDB].[Ice].[ChgLog] 
) a
Where rn=1 
) cl
  ON a.JobNum=cl.Key1 and a.ForeignKey2=cl.Key2

  WHERE a.Jobnum like '2170022%' and a.Company = 'Solar'
  Order By DateStampedOn DESC

(30 rows affected)

Total execution time: 00:00:03.886

Company,JobComplete,IssuedComplete,JobNum,AssemblySeq,MtlSeq,PartNum,Description,QtyPer,RequiredQty,IUM,LeadTime,RelatedOperation,MtlBurRate,EstMtlBurUnitCost,EstUnitCost,IssuedQty,TotalCost,MtlBurCost,ReqDate,WarehouseCode,SalvagePartNum,SalvageDescription,SalvageQtyPer,SalvageUM,SalvageMtlBurRate,SalvageUnitCredit,SalvageEstMtlBurUnitCredit,SalvageQtyToDate,SalvageCredit,SalvageMtlBurCredit,MfgComment,VendorNum,PurPoint,BuyIt,Ordered,PurComment,BackFlush,EstScrap,EstScrapType,FixedQty,FindNum,RevisionNum,SndAlrtCmpl,RcvInspectionReq,Plant,Direct,MaterialMtlCost,MaterialLabCost,MaterialSubCost,MaterialBurCost,SalvageMtlCredit,SalvageLbrCredit,SalvageBurCredit,SalvageSubCredit,APSAddResType,CallNum,CallLine,ProdCode,UnitPrice,BillableUnitPrice,DocBillableUnitPrice,ResReasonCode,PricePerCode,Billable,ShippedQty,DocUnitPrice,QtyStagedToDate,AddedMtl,MiscCharge,MiscCode,SCMiscCode,RFQNeeded,RFQVendQuotes,RFQNum,RFQLine,RFQStat,AnalysisCode,GlbRFQ,WhseAllocFlag,WIReqDate,Rpt1BillableUnitPrice,Rpt2BillableUnitPrice,Rpt3BillableUnitPrice,Rpt1UnitPrice,Rpt2UnitPrice,Rpt3UnitPrice,BaseRequiredQty,BaseUOM,Weight,WeightUOM,ReqRefDes,BasePartNum,BaseRevisionNum,SelectForPicking,StagingWarehouseCode,StagingBinNum,PickError,EstMtlUnitCost,EstLbrUnitCost,EstBurUnitCost,EstSubUnitCost,SalvageEstMtlUnitCredit,SalvageEstLbrUnitCredit,SalvageEstBurUnitCredit,SalvageEstSubUnitCredit,LoanedQty,BorrowedQty,ReassignSNAsm,GeneralPlanInfo,EstStdDescription,PricingUOM,RemovedFromPlan,IsPOCostingMaintained,EstStdType,POCostingFactor,PlannedQtyPerUnit,POCostingDirection,POCostingUnitVal,GroupSeq,SysRevID,SysRowID,OrigStructTag,OrigGroupSeq,ShowStatusIcon,ContractID,LinkToContract,StagingLotNum,PCLinkRemoved,ExternalMESSyncRequired,ExternalMESLastSync,LocationView,AttributeSetID,PlanningNumberOfPieces,SalvageAttributeSetID,SalvagePlanningNumberOfPieces,SalvagePlanningAttributeSetID,ForeignKey2,DateStampedOn,Key1,Key2,LogText,rn
SOLAR,0,1,2170022-5-1,0,260,260000,Sherwin Williams,1.00000000,1.00000000,EA,0,0,0.00000,0.00000,0.00000,1.00000000,1168.860,0.000,2021-03-12,,,,0.00000000,,0.00000,0.00000,0.00000,0.00000000,0.000,0.000,,0,,0,0,,0,0.00,%,1,,,0,0,MfgSys,0,1168.860,0.000,0.000,0.000,0.000,0.000,0.000,0.000,M,0,0,,0.00000,0.00000,0.00000,,E,0,0.00000000,0.00000,0.00000000,0,1,MISC,,0,0,0,0,,,0,0,2021-03-12,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000000,EA,0.00000,,0,,,0,,,,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00,0.00,0,,,,0,0,0,0.00000,0.00000,0,0.00000,0,0x0000000325D4E93D,e5ce3385-173f-413b-86f4-09a5beba3202,,0,,,0,,0,1,NULL,0,0,0,0,0,0,2170022-5-1~0~260,2022-07-14,2170022-5-1,2170022-5-1~0~260,Kbhughett 11:43:55 New Record,1
SOLAR,0,1,2170022-5-1,0,250,250000,Tri County Asphalt,1.00000000,1.00000000,EA,0,0,0.00000,0.00000,0.00000,1.00000000,15000.000,0.000,2021-03-12,,,,0.00000000,,0.00000,0.00000,0.00000,0.00000000,0.000,0.000,,0,,0,0,,0,0.00,%,1,,,0,0,MfgSys,0,15000.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,M,0,0,,0.00000,0.00000,0.00000,,E,0,0.00000000,0.00000,0.00000000,0,1,MISC,,0,0,0,0,,,0,0,2021-03-12,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000000,EA,0.00000,,0,,,0,,,,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00,0.00,0,,,,0,0,0,0.00000,0.00000,0,0.00000,0,0x0000000325D4E986,4dc1280a-757d-4c31-8c63-f30c574ed513,,0,,,0,,0,1,NULL,0,0,0,0,0,0,2170022-5-1~0~250,2022-07-11,2170022-5-1,2170022-5-1~0~250,Kbhughett 16:32:29 New Record,1
SOLAR,0,1,2170022-5-1,0,240,240000,Gaddis & Son Inc,1.00000000,1.00000000,EA,0,0,0.00000,0.00000,40000.00000,1.00000000,9700.000,0.000,2021-03-12,,,,0.00000000,,0.00000,0.00000,0.00000,0.00000000,0.000,0.000,,0,,0,0,,0,0.00,%,1,,,0,0,MfgSys,0,9700.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,M,0,0,,0.00000,0.00000,0.00000,,E,0,0.00000000,0.00000,0.00000000,0,1,MISC,,0,0,0,0,,,0,0,2021-03-12,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000000,EA,0.00000,,0,,,0,,,,40000.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000

In [9]:
Select *
From
(Select DateStampedOn,
        Key1,
        Key2,
        LogText,
        ROW_NUMBER() OVER(Partition by Key2 Order by DateStampedON Desc) rn
    From [ERPDB].[Ice].[ChgLog] 
) a
Where rn=1 And Key1 like '2170022%'
Order By DateStampedOn Desc


(766 rows affected)

Total execution time: 00:00:03.595

DateStampedOn,Key1,Key2,LogText,rn
2022-07-14,2170022-5-1,2170022-5-1~0~260,Kbhughett 11:43:55 New Record,1
2022-07-11,2170022,2170022~147,Kbhughett 09:32:06 New Record Kbhughett 09:41:10 OpenLine: True -> False,1
2022-07-11,2170022,2170022~148,Kbhughett 09:32:08 New Record Kbhughett 09:41:10 OpenLine: True -> False,1
2022-07-11,2170022-5-1,2170022-5-1~0~250,Kbhughett 16:32:29 New Record,1
2022-07-06,2170022-5-1,2170022-5-1~0~140,Adamw 14:59:46 EstUnitCost: 120000.00000 -> 214500.00000,1
2022-07-06,2170022-5-1,2170022-5-1~0~190,Adamw 10:57:01 EstUnitCost: 28000.00000 -> 7181.00000,1
2022-07-06,2170022-5-1,2170022-5-1~0~210,Adamw 10:57:07 EstUnitCost: 10000.00000 -> 0,1
2022-07-06,2170022-5-1,2170022-5-1~0~220,Adamw 10:57:09 EstUnitCost: 5000.00000 -> 0,1
2022-07-06,2170022-5-1,2170022-5-1~0~240,Adamw 10:57:32 EstUnitCost: 9700.00000 -> 45519.00000 Adamw 14:59:53 EstUnitCost: 45519.00000 -> 40000.00000,1
2022-07-05,2170022-3-1,2170022-3-1~0~290,KWILLIAMS 13:01:46 New Record,1


- **John PartTran and JobHead to get calculated Change date for Job Assembly Seq**

In [3]:
SELECT jh.*, pt.*,
(case when
 pt.pt_TranDate >= jh.jh_CreateDate
 then
 pt.pt_TranDate
 when
 jh.jh_CreateDate >= jh.jh_ClosedDate
 then
 jh.jh_CreateDate
 else
 jh.jh_ClosedDate
 end) as [Calculated_ChangeDate]
FROM

(SELECT
    Company as pt_Company,
    JobNum as pt_JobNum,
    AssemblySeq as pt_AssemblySeq,
    JobSeq as pt_JobSeq,
    TranDate as pt_TranDate,
    PartNum as pt_PartNum
    FROM [ERPDB].[Erp].[PartTran]
) as pt
Left Join
(SELECT 
	Company as jh_Company, 
    ClosedDate as jh_ClosedDate, 
	JobNum as jh_JobNum,
	CreateDate as jh_CreateDate
    FROM [ERPDB].[Erp].[JobHead]
 ) as jh
    on pt.pt_Company = jh.jh_Company AND pt.pt_JobNum = jh.jh_JobNum
  WHERE pt.pt_Jobnum like '2170022%' and pt.pt_Company = 'Solar'
  Order By Calculated_ChangeDate DESC

(332 rows affected)

Total execution time: 00:00:00.113

jh_Company,jh_ClosedDate,jh_JobNum,jh_CreateDate,pt_Company,pt_JobNum,pt_AssemblySeq,pt_JobSeq,pt_TranDate,pt_PartNum,Calculated_ChangeDate
SOLAR,NULL,2170022-5-1,2021-02-25,SOLAR,2170022-5-1,0,260,2022-07-14,,2022-07-14
SOLAR,NULL,2170022-5-1,2021-02-25,SOLAR,2170022-5-1,0,260,2022-07-14,,2022-07-14
SOLAR,NULL,2170022-5-1,2021-02-25,SOLAR,2170022-5-1,0,140,2022-07-14,,2022-07-14
SOLAR,NULL,2170022-5-1,2021-02-25,SOLAR,2170022-5-1,0,250,2022-07-14,,2022-07-14
SOLAR,NULL,2170022-5-1,2021-02-25,SOLAR,2170022-5-1,0,170,2022-07-12,,2022-07-12
SOLAR,NULL,2170022-5-1,2021-02-25,SOLAR,2170022-5-1,0,170,2022-07-12,17000,2022-07-12
SOLAR,NULL,2170022-5-1,2021-02-25,SOLAR,2170022-5-1,0,20,2022-07-12,02000,2022-07-12
SOLAR,NULL,2170022-5-1,2021-02-25,SOLAR,2170022-5-1,0,40,2022-07-11,,2022-07-11
SOLAR,NULL,2170022-5-1,2021-02-25,SOLAR,2170022-5-1,0,140,2022-07-11,,2022-07-11
SOLAR,NULL,2170022-5-1,2021-02-25,SOLAR,2170022-5-1,0,250,2022-07-11,,2022-07-11


In [7]:
--Get the most recent calculated_change date by the Job Assembly Seq

Select  *
From 
(Select 
    f.Company,
    f.JobNum,
    f.AssemblySeq,
    f.JobSeq,
    f.PartNum,
    f.TranDate,
    f.CreateDate,
    f.CloseDate,
    f.ForeignKey2,
    f.Calculated_ChangeDate,
    ROW_NUMBER() OVER(PARTITION By ForeignKey2 ORDER BY f.Calculated_ChangeDate DESC) rn
FROM
    (SELECT 
        pt.Company as Company, 
        pt.JobNum as JobNum,
        pt.AssemblySeq as AssemblySeq,
        pt.JobSeq as JobSeq,
        pt.TranDate as TranDate,
        pt.PartNum as PartNum,
        jh.CreateDate as CreateDate,
        jh.ClosedDate as CloseDate,
        pt.ForeignKey2 as ForeignKey2,
        (case when
        pt.TranDate >= jh.CreateDate
        then
        pt.TranDate
        when
        jh.CreateDate >= jh.ClosedDate
        then
        jh.CreateDate
        else
        jh.ClosedDate
        end) as Calculated_ChangeDate
            FROM
            (SELECT
                Company,
                JobNum,
                AssemblySeq,
                JobSeq,
                TranDate,
                PartNum,
                CONCAT(JobNum,'~',AssemblySeq,'~',JobSeq) as ForeignKey2
                FROM [ERPDB].[Erp].[PartTran]
            ) as pt
            Left Join
            (SELECT 
                Company, 
                ClosedDate, 
                JobNum,
                CreateDate 
                FROM [ERPDB].[Erp].[JobHead]
            ) as jh
                on pt.Company = jh.Company AND pt.JobNum = jh.JobNum
            WHERE pt.Jobnum like '2170022%' and pt.Company = 'Solar'
    ) f 
) b
 Where rn = 1
 Order By Calculated_ChangeDate Desc

(26 rows affected)

Total execution time: 00:00:00.093

Company,JobNum,AssemblySeq,JobSeq,PartNum,TranDate,CreateDate,CloseDate,ForeignKey2,Calculated_ChangeDate,rn
SOLAR,2170022-5-1,0,140,,2022-07-14,2021-02-25,NULL,2170022-5-1~0~140,2022-07-14,1
SOLAR,2170022-5-1,0,250,,2022-07-14,2021-02-25,NULL,2170022-5-1~0~250,2022-07-14,1
SOLAR,2170022-5-1,0,260,,2022-07-14,2021-02-25,NULL,2170022-5-1~0~260,2022-07-14,1
SOLAR,2170022-5-1,0,20,02000,2022-07-12,2021-02-25,NULL,2170022-5-1~0~20,2022-07-12,1
SOLAR,2170022-5-1,0,170,,2022-07-12,2021-02-25,NULL,2170022-5-1~0~170,2022-07-12,1
SOLAR,2170022-2-1,0,10,MFG Structure,2022-07-11,2021-02-25,NULL,2170022-2-1~0~10,2022-07-11,1
SOLAR,2170022-4-1,0,10,MFG Structure,2022-07-11,2021-02-25,NULL,2170022-4-1~0~10,2022-07-11,1
SOLAR,2170022-5-1,0,40,,2022-07-11,2021-02-25,NULL,2170022-5-1~0~40,2022-07-11,1
SOLAR,2170022-5-1,0,160,,2022-07-06,2021-02-25,NULL,2170022-5-1~0~160,2022-07-06,1
SOLAR,2170022-5-1,0,200,,2022-07-06,2021-02-25,NULL,2170022-5-1~0~200,2022-07-06,1


This is to test the joins with the job assembly table

In [2]:
SELECT a.*, jh.*, pt.*,
(case when
 pt.pt_TranDate >= jh.jh_CreateDate
 then
 pt.pt_TranDate
 when
 jh.jh_CreateDate >= jh.jh_ClosedDate
 then
 jh.jh_CreateDate
 else
 jh.jh_ClosedDate
 end) as [Calculated_ChangeDate]
FROM
(
    SELECT JobMtl.*,
    CONCAT(JobNum,'~',AssemblySeq,'~',MtlSeq) as ForeignKey2
    FROM [ERPDB].[Erp].[JobMtl]
) a

LEFT JOIN 
(SELECT 
	Company as jh_Company, 
    ClosedDate as jh_ClosedDate, 
	JobNum as jh_JobNum,
	CreateDate as jh_CreateDate
    FROM [ERPDB].[Erp].[JobHead]
 ) as jh
	on a.Company = jh.jh_Company and a.JobNum = jh.jh_JobNum

LEFT JOIN
(SELECT
    Company as pt_Company,
    JobNum as pt_JobNum,
    AssemblySeq as pt_AssemblySeq,
    JobSeq as pt_JobSeq,
    TranDate as pt_TranDate,
    PartNum as pt_PartNum
    FROM [ERPDB].[Erp].[PartTran]
) as pt
    on a.Company = pt.pt_Company AND a.JobNum = pt.pt_JobNum AND a.AssemblySeq = pt.pt_AssemblySeq AND a.PartNum = pt.pt_PartNum
  WHERE a.Jobnum like '2170022%' and a.Company = 'Solar'
  Order By Calculated_ChangeDate DESC

(62 rows affected)

Total execution time: 00:00:00.270

Company,JobComplete,IssuedComplete,JobNum,AssemblySeq,MtlSeq,PartNum,Description,QtyPer,RequiredQty,IUM,LeadTime,RelatedOperation,MtlBurRate,EstMtlBurUnitCost,EstUnitCost,IssuedQty,TotalCost,MtlBurCost,ReqDate,WarehouseCode,SalvagePartNum,SalvageDescription,SalvageQtyPer,SalvageUM,SalvageMtlBurRate,SalvageUnitCredit,SalvageEstMtlBurUnitCredit,SalvageQtyToDate,SalvageCredit,SalvageMtlBurCredit,MfgComment,VendorNum,PurPoint,BuyIt,Ordered,PurComment,BackFlush,EstScrap,EstScrapType,FixedQty,FindNum,RevisionNum,SndAlrtCmpl,RcvInspectionReq,Plant,Direct,MaterialMtlCost,MaterialLabCost,MaterialSubCost,MaterialBurCost,SalvageMtlCredit,SalvageLbrCredit,SalvageBurCredit,SalvageSubCredit,APSAddResType,CallNum,CallLine,ProdCode,UnitPrice,BillableUnitPrice,DocBillableUnitPrice,ResReasonCode,PricePerCode,Billable,ShippedQty,DocUnitPrice,QtyStagedToDate,AddedMtl,MiscCharge,MiscCode,SCMiscCode,RFQNeeded,RFQVendQuotes,RFQNum,RFQLine,RFQStat,AnalysisCode,GlbRFQ,WhseAllocFlag,WIReqDate,Rpt1BillableUnitPrice,Rpt2BillableUnitPrice,Rpt3BillableUnitPrice,Rpt1UnitPrice,Rpt2UnitPrice,Rpt3UnitPrice,BaseRequiredQty,BaseUOM,Weight,WeightUOM,ReqRefDes,BasePartNum,BaseRevisionNum,SelectForPicking,StagingWarehouseCode,StagingBinNum,PickError,EstMtlUnitCost,EstLbrUnitCost,EstBurUnitCost,EstSubUnitCost,SalvageEstMtlUnitCredit,SalvageEstLbrUnitCredit,SalvageEstBurUnitCredit,SalvageEstSubUnitCredit,LoanedQty,BorrowedQty,ReassignSNAsm,GeneralPlanInfo,EstStdDescription,PricingUOM,RemovedFromPlan,IsPOCostingMaintained,EstStdType,POCostingFactor,PlannedQtyPerUnit,POCostingDirection,POCostingUnitVal,GroupSeq,SysRevID,SysRowID,OrigStructTag,OrigGroupSeq,ShowStatusIcon,ContractID,LinkToContract,StagingLotNum,PCLinkRemoved,ExternalMESSyncRequired,ExternalMESLastSync,LocationView,AttributeSetID,PlanningNumberOfPieces,SalvageAttributeSetID,SalvagePlanningNumberOfPieces,SalvagePlanningAttributeSetID,ForeignKey2,jh_Company,jh_ClosedDate,jh_JobNum,jh_CreateDate,pt_Company,pt_JobNum,pt_AssemblySeq,pt_JobSeq,pt_TranDate,pt_PartNum,Calculated_ChangeDate
SOLAR,0,1,2170022-5-1,0,20,02000,EXPENSE REPORT,1.00000000,1.00000000,EA,0,0,0.00000,0.00000,15000.00000,1.00000000,13407.300,0.000,2021-03-12,,,,0.00000000,,0.00000,0.00000,0.00000,0.00000000,0.000,0.000,,0,,0,0,,0,0.00,%,1,,,0,0,MfgSys,0,13407.300,0.000,0.000,0.000,0.000,0.000,0.000,0.000,M,0,0,,0.00000,0.00000,0.00000,,E,0,0.00000000,0.00000,0.00000000,0,1,MISC,,0,0,0,0,,,0,0,2021-03-12,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000000,EA,0.00000,,0,,,0,,,,15000.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00,0.00,0,,,,0,0,0,0.00000,0.00000,0,0.00000,0,0x000000032469E1B3,eeb673ec-a1bb-4a27-ab74-d2fdbd68765b,,0,Complete,,0,,0,1,NULL,0,0,0,0,0,0,2170022-5-1~0~20,SOLAR,NULL,2170022-5-1,2021-02-25,SOLAR,2170022-5-1,0,20,2022-07-12,02000,2022-07-12
SOLAR,0,1,2170022-5-1,0,170,17000,LBFNY,1.00000000,1.00000000,EA,0,0,0.00000,0.00000,417206.27000,1.00000000,190881.320,0.000,2021-03-12,,,,0.00000000,,0.00000,0.00000,0.00000,0.00000000,0.000,0.000,,0,,0,0,,0,0.00,%,1,,,0,0,MfgSys,0,190881.320,0.000,0.000,0.000,0.000,0.000,0.000,0.000,M,0,0,,0.00000,0.00000,0.00000,,E,0,0.00000000,0.00000,0.00000000,0,1,MISC,,0,0,0,0,,,0,0,2021-03-12,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000000,EA,0.00000,,0,,,0,,,,417206.27000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00,0.00,0,,,,0,0,0,0.00000,0.00000,0,0.00000,0,0x000000032469E0DB,a7ff1e53-d1ac-4603-9499-3708fb9b2770,,0,Complete,,0,,0,1,NULL,0,0,0,0,0,0,2170022-5-1~0~170,SOLAR,NULL,2170022-5-1,2021-02-25,SOLAR,2170022-5-1,0,170,2022-07-12,17000,2022-07-12
SOLAR,0,1,2170022-5-1,0,170,17000,LBFNY,1.00000000,1.00000000,EA,0,0,0.00000,0.00000,417206.27000,1.00000000,190881.320,0.000,2021-03-12,,,,0.00000000,,0.00000,0.00000,0.00000,0.00000000,0.000,0.000,,0,,0,0,,0,0.00,%,1,,,0,0,MfgSys,0,190881.320,0.000,0.000,0.000,0.000,0.000,0.000,0.000,M,0,0,,0.00000,0.00000,0.00000,,E,0,0.00000000,0.00000,0.00000000,0,1,MISC,,0,0,0,0,,,0,0,2021-03-12,0.000

This is the base Job Material table to compare to the joins

In [1]:
SELECT TOP (1000) [Company]
      ,[JobComplete]
      ,[IssuedComplete]
      ,[JobNum]
      ,[AssemblySeq]
      ,[MtlSeq]
      ,[PartNum]
      ,[Description]
      ,[QtyPer]
      ,[RequiredQty]
      ,[IUM]
      ,[EstUnitCost]
      ,[IssuedQty]
      ,[TotalCost]
      ,[MtlBurCost]
      ,[ReqDate]
      ,[Plant]
      ,[MaterialMtlCost]
      ,[MaterialLabCost]
      ,[MaterialSubCost]
      ,[MaterialBurCost]
      ,[ProdCode]
      ,[UnitPrice]
      ,[WIReqDate]
      ,[EstMtlUnitCost]
      ,[EstLbrUnitCost]
      ,[EstBurUnitCost]
      ,[EstSubUnitCost]
      ,[SysRevID]
      ,[SysRowID]

  FROM [ERPDB].[Erp].[JobMtl]
  WHERE JobNum like '2170022%'
  and Company = 'Solar'

(30 rows affected)

Total execution time: 00:00:00.068

Company,JobComplete,IssuedComplete,JobNum,AssemblySeq,MtlSeq,PartNum,Description,QtyPer,RequiredQty,IUM,EstUnitCost,IssuedQty,TotalCost,MtlBurCost,ReqDate,Plant,MaterialMtlCost,MaterialLabCost,MaterialSubCost,MaterialBurCost,ProdCode,UnitPrice,WIReqDate,EstMtlUnitCost,EstLbrUnitCost,EstBurUnitCost,EstSubUnitCost,SysRevID,SysRowID
SOLAR,0,1,2170022-13-1,0,10,BONDS,BONDS,1.00000000,1.00000000,EA,0.00000,1.00000000,70949.000,0.000,2021-03-12,MfgSys,70949.000,0.000,0.000,0.000,BND,0.00000,2021-03-12,0.00000,0.00000,0.00000,0.00000,0x00000001DC40852E,9142ce14-c2fd-48a5-ad59-7590408f163a
SOLAR,0,1,2170022-2-1,0,10,MFG MATERIAL,MFG MATERIAL,1.00000000,1.00000000,EA,0.00000,1.00000000,3869428.000,0.000,2021-03-12,MfgSys,3869428.000,0.000,0.000,0.000,MFG,0.00000,2021-03-12,0.00000,0.00000,0.00000,0.00000,0x0000000322ACF7E0,b749bb08-cd50-41e2-b8b5-ac9aa7ee8567
SOLAR,0,1,2170022-4-1,0,10,FREIGHT,FREIGHT,1.00000000,1.00000000,EA,0.00000,1.00000000,244034.000,0.000,2021-03-12,MfgSys,244034.000,0.000,0.000,0.000,FRT,0.00000,2021-03-12,0.00000,0.00000,0.00000,0.00000,0x0000000322ACF7E5,554ae377-75f3-474b-b6ac-5ca92d037c19
SOLAR,0,1,2170022-4-1,0,20,02000,FED EX,1.00000000,1.00000000,EA,0.00000,1.00000000,388.810,0.000,2021-03-12,MfgSys,388.810,0.000,0.000,0.000,,0.00000,2021-03-12,0.00000,0.00000,0.00000,0.00000,0x000000031D1EFB87,773438e1-66c9-42c2-a7b0-dee0520ea6b3
SOLAR,0,1,2170022-5-1,0,10,01000,EMH&T,1.00000000,1.00000000,EA,23701.00000,1.00000000,17670.510,0.000,2021-03-12,MfgSys,17670.510,0.000,0.000,0.000,,0.00000,2021-03-12,23701.00000,0.00000,0.00000,0.00000,0x000000030707EC0F,b2d04085-5dfd-48fa-8c99-88629b89aaf4
SOLAR,0,1,2170022-5-1,0,20,02000,EXPENSE REPORT,1.00000000,1.00000000,EA,15000.00000,1.00000000,13407.300,0.000,2021-03-12,MfgSys,13407.300,0.000,0.000,0.000,,0.00000,2021-03-12,15000.00000,0.00000,0.00000,0.00000,0x000000032469E1B3,eeb673ec-a1bb-4a27-ab74-d2fdbd68765b
SOLAR,0,1,2170022-5-1,0,30,03000,Precision Drilling,1.00000000,1.00000000,EA,1209600.00000,1.00000000,709200.000,0.000,2021-03-12,MfgSys,709200.000,0.000,0.000,0.000,,0.00000,2021-03-12,1209600.00000,0.00000,0.00000,0.00000,0x00000002F55EA2AD,651ed0c6-4f6e-4d73-9161-fb0dd9c77a82
SOLAR,0,1,2170022-5-1,0,40,04000,FC Contracting,1.00000000,1.00000000,EA,1110419.00000,1.00000000,1115819.000,0.000,2021-03-12,MfgSys,1115819.000,0.000,0.000,0.000,,0.00000,2021-03-12,1110419.00000,0.00000,0.00000,0.00000,0x00000003233C73AE,08b6e033-4f7f-4291-8b07-3a1608dbd811
SOLAR,0,1,2170022-5-1,0,50,05000,William Scotsman Trailer,1.00000000,1.00000000,EA,10500.00000,1.00000000,6202.420,0.000,2021-03-12,MfgSys,6202.420,0.000,0.000,0.000,,0.00000,2021-03-12,10500.00000,0.00000,0.00000,0.00000,0x000000031DA5E9DE,712f14d3-482a-459f-b55c-4d3c26871ecf
SOLAR,0,1,2170022-5-1,0,60,06000,Contingency,1.00000000,1.00000000,EA,0.00000,1.00000000,0.000,0.000,2021-03-12,MfgSys,0.000,0.000,0.000,0.000,,0.00000,2021-03-12,0.00000,0.00000,0.00000,0.00000,0x00000002FFA3F54B,d5398e56-41db-4ed7-b411-5886e701d21f
